In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import os
import re
import numpy as np
import pandas as pd
import collections
import gc

In [ ]:
x_name = os.path.join(os.getcwd(), 'SCDC_Track1_2차', f"{'cst_feat_jan'}.csv")
df_x = pd.read_csv(x_name, index_col=0)

y_name = os.path.join(os.getcwd(), 'SCDC_Track1_2차', f"{'train'}.csv")
df_y = pd.read_csv(y_name, index_col=0)

v_name = os.path.join(os.getcwd(), 'SCDC_Track1_2차', f"{'val'}.csv")
df_v = pd.read_csv(v_name, index_col=0)

df_yv = pd.concat([df_y, df_v])
df = pd.merge(df_x, df_yv, on='cst_id_di')

In [ ]:
X = df.drop(columns = ['MRC_ID_DI'], axis=1)
y = df['MRC_ID_DI']

In [ ]:
X.info(memory_usage='deep')

In [ ]:
len(X.columns)*0.5

In [ ]:
import sklearn.feature_selection
gc.collect()
select = sklearn.feature_selection.SelectKBest(k=113) #113
selected_features = select.fit(X, y)
indices_selected = selected_features.get_support(indices=True)
colnames_selected = [X.columns[i] for i in indices_selected]

X = X[colnames_selected]

In [ ]:
from itertools import combinations
from sklearn.preprocessing import PolynomialFeatures

X = X.astype(np.float16)
combos = list(combinations(list(X.columns), 2))
colnames = list(X.columns) + ['_'.join(x) for x in combos]

poly = PolynomialFeatures(interaction_only=True, include_bias=False)
X = poly.fit_transform(X)
X = pd.DataFrame(X)
X.columns = colnames

gc.collect()

noint_indicies = [i for i, x in enumerate(list((X == 0).all())) if x]
X = X.drop(X.columns[noint_indicies], axis = 1)

In [ ]:
temp = X
temp['cst_id_di'] = y.index
temp = temp.set_index('cst_id_di')
temp['MRC_ID_DI'] = y

X = temp.drop(columns = ['MRC_ID_DI'], axis=1)
y = temp['MRC_ID_DI'].astype(np.int8)
gc.collect()

In [ ]:
import sklearn.feature_selection

select = sklearn.feature_selection.SelectKBest(k=331)
selected_features = select.fit(X, y)
indices_selected = selected_features.get_support(indices=True)
colnames_selected = [X.columns[i] for i in indices_selected]

X = X[colnames_selected]

gc.collect()

In [ ]:
temp = X
temp['cst_id_di'] = y.index
temp = temp.set_index('cst_id_di')
temp['MRC_ID_DI'] = y

temp_0 = temp[temp['MRC_ID_DI'] == 0].sample(frac=1)
temp_1 = temp[temp['MRC_ID_DI'] == 1].sample(frac=1)
temp_2 = temp[temp['MRC_ID_DI'] == 2].sample(frac=1)
temp_3 = temp[temp['MRC_ID_DI'] == 3].sample(frac=1)
temp_4 = temp[temp['MRC_ID_DI'] == 4].sample(frac=1)
temp_5 = temp[temp['MRC_ID_DI'] == 5].sample(frac=1)
temp_6 = temp[temp['MRC_ID_DI'] == 6].sample(frac=1)
temp_7 = temp[temp['MRC_ID_DI'] == 7].sample(frac=1)
temp_8 = temp[temp['MRC_ID_DI'] == 8].sample(frac=1)
temp_9 = temp[temp['MRC_ID_DI'] == 9].sample(frac=1)
temp_10 = temp[temp['MRC_ID_DI'] == 10].sample(frac=1)

sample_size = min(len(temp_0),len(temp_1), len(temp_2), len(temp_3), len(temp_4), len(temp_5), len(temp_6), len(temp_7),
                 len(temp_8), len(temp_9), len(temp_10))

temp_f = pd.concat([temp_0.head(sample_size), temp_1.head(sample_size), temp_2.head(sample_size), temp_3.head(sample_size),
                    temp_4.head(sample_size), temp_5.head(sample_size), temp_6.head(sample_size), temp_7.head(sample_size), 
                    temp_8.head(sample_size), temp_9.head(sample_size), temp_10.head(sample_size)]).sample(frac=1)

X = temp_f.drop(columns = ['MRC_ID_DI'], axis=1)
y = temp_f['MRC_ID_DI']

gc.collect()

In [ ]:
X[X.columns] = (X[X.columns]+1)/2*255

In [ ]:
nth_img = 'image10'
img_dir = os.path.join(os.getcwd(), nth_img)
if not os.path.exists(img_dir):
    os.makedirs(img_dir)

In [ ]:
repetitions = 331
p = 331

from PIL import Image
import numpy as np
for ind, k in enumerate(y.index):
    print(ind)
    data = np.zeros((p, p, 3), dtype=np.float16)
    
    an_array = X.iloc[ind].values
    row_array = np.tile(an_array, (repetitions, 1))
    col_array = np.transpose([an_array] * repetitions)
    final_array = (row_array + col_array) / 2
    final_array = np.repeat(final_array.flatten(), 3)
    data = final_array.reshape((p,p, 3))
    
    img = Image.fromarray(data.astype(np.uint8), 'RGB')
    
    img_name = os.path.join(os.getcwd(), nth_img, str(k) + '.png')
    img.save(img_name)
    
    gc.collect()